<a href="https://colab.research.google.com/github/dezlorator1/DSY/blob/main/Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from tensorflow.keras.datasets import mnist         # библиотека базы выборок Mnist
from tensorflow import keras
from tensorflow.keras.layers import Dense, Flatten
from PIL import Image
from google.colab import drive
from google.colab import files

ПОДГОТОВ ОЧКА КАРТИНОК


In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
img = mpimg.imread('/content/drive/MyDrive/img/CutMoscow.jpg')

plt.figure()
plt.imshow(img)

In [ ]:
chopsize = 50
infile = r"/content/drive/MyDrive/img/CutMoscow.jpg"
outfile = r"/content/drive/MyDrive/img/cut_img_color/"
img = Image.open(infile)
width, height = img.size
count = 0 ;
# Save Chops of original image
for x0 in range(0, width, chopsize):
   for y0 in range(0, height, chopsize):
      box = (x0, y0,
             x0+chopsize if x0+chopsize <  width else  width - 1,
             y0+chopsize if y0+chopsize < height else height - 1)
      #print('%s %s' % (infile, box))
      img.crop(box).save(outfile + f'{count}' + 'c.jpg', "JPEG")
      count = count + 1

In [ ]:
files.upload()

ЗАГОТОВКА МАССИВОВ ДАННЫХ

In [ ]:
cut_black = np.array([])
#black = np.array([])
for i in range(2):
  img = mpimg.imread('/content/drive/MyDrive/img/cut_img_black/' + f'{i}' + '.jpg')
  #black = np.append(black, img)
  cut_black = np.append(cut_black, img)
  print(cut_black)
  #cut_black.append(img)

[ 7.  7.  7. ... 25. 25. 25.]
[ 7.  7.  7. ... 11. 11. 11.]


In [ ]:
cut_color = np.array([])
for i in range(960):
    img = mpimg.imread('/content/drive/MyDrive/img/cut_img_color/' + f'{i}' + 'c.jpg')
    cut_color = np.append(cut_color, img)
    #cut_color.append(img)

In [ ]:
print(cut_black[0])

In [ ]:
plt.figure()
plt.imshow(cut_black[0])
#plt.figure()
#plt.imshow(cut_color[0])

In [ ]:
for i in range(960):
  cut_black[i] = cut_black[i] / 255
  cut_color[i] = cut_color[i] / 255

НЕЙРОНКА


In [ ]:
model = keras.Sequential([
    Flatten(input_shape=(50, 50, 3)),
    Dense(2000, activation='relu'),
    Dense(500, activation='relu'),
    Dense(7500, activation='softmax')
])

In [ ]:
print(model.summary()) 

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_2 (Flatten)          (None, 2500)              0         
_________________________________________________________________
dense_6 (Dense)              (None, 2000)              5002000   
_________________________________________________________________
dense_7 (Dense)              (None, 500)               1000500   
_________________________________________________________________
dense_8 (Dense)              (None, 2500)              1252500   
Total params: 7,255,000
Trainable params: 7,255,000
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
model.compile(optimizer='adam',
             loss='categorical_crossentropy',
             metrics=['accuracy'])

In [ ]:
model.fit(cut_black, cut_color, batch_size=32, epochs=5, validation_split=0.15)

Epoch 1/5


ValueError: ignored